# Hidden markov model generalized to 2 parenting relationship (also ngram model, n=3)

In [20]:
# P(X_i|X_i-1,X_i-2)

In [21]:
import nltk
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle 
import json
import os
from pathlib import Path
from IPython.display import Image, Audio
from music21 import note , chord , stream , instrument , converter   
import mido

In [22]:
# create a dict with the ngram model, it receives a list with the samples as string
ngram_dict = {}

def create_trigram_dict(corpus):
    n = 3
    ngrams = nltk.ngrams(corpus, n)
    
    for grams in ngrams:
        dict_key = grams[:-1][0] + " " + grams[:-1][1]
        if dict_key in ngram_dict:
            ngram_dict[dict_key].append(grams[-1])
        else:
            ngram_dict[dict_key] = []
            ngram_dict[dict_key].append(grams[-1])

In [23]:
def generate_trigram(seed, samples, ngram_dict):
    output = seed  
    for i in range(samples):
        # When it reaches the last prefix, there is no suffix, so end
        try:
            new_sample = random.choice(ngram_dict[seed])
        except:
            return output
        output += " " + new_sample
        seed = seed.split(" ")[1] + " " + new_sample

    return output

In [24]:
songs = []
for filename in os.listdir("dataset/midi_songs"):
    if filename.endswith(".mid"): 
        #print(filename)
        s = 'dataset/midi_songs/'+filename
        #song = mido.MidiFile(s)
        conv = converter.parse(s)
        songs.append(conv)
        continue
    else:
        continue


In [25]:
notes = []
for s in songs:
    notes_to_parse = s.flat.notes
    notes.append(notes_to_parse)
    

In [26]:
notes[:5]

[<music21.stream.iterator.StreamIterator for Score:0x2968f35e0 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x1052e3670 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x296a96cd0 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x2968f3940 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x296b6cc40 @:0>]

In [127]:
#                                      song1  song2
#generate list of list of notes --> [[A,F,C],[D,G,B]]
listoflists = []
for n in notes:
    notes_demo = []
    for element in n:
        # if the element is a Note , then store it's Pitch
        if isinstance(element , note.Note):
            notes_demo.append(str(element.pitch))
        
        elif isinstance(element , chord.Chord):
            notes_demo.append('+'.join(str(n) for n in element.normalOrder))
            
    listoflists.append(notes_demo)
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<music21.stream.iterator.StreamIterator for Score:0x294688940 @:364>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:366>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:367>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:369>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:371>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:373>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:377>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:379>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:381>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:383>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:385>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:386>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:387>
<music21.stream.iterator.StreamIterator for Score:0x294688940 @:388>
<music21.stream.iterator.StreamIte

In [28]:
# converting into a single string each song in listoflists
final = []
for song in listoflists:
    l = song
    melody = ''
    for i in l:
        melody = melody + ' ' + i
    final.append(melody)
    melody = ''

In [29]:
dictionaries = []
for progression in final:
    ngram_dict = {}
    create_trigram_dict(progression.split(" "))
    dictionaries.append(ngram_dict)
    

In [30]:
#dictionaries[6]

In [31]:
#generate melody based on trigram model and pattern recognition on ngram_dict
#generated_melody = generate_trigram("B-3 D4", 100, dictionaries[6])
#print(generated_melody)

In [32]:
#create suitable format for playing midi file concatenating the string with the notation and tempo
#generated_melody = "tinynotation: 4/4" + generated_melody

In [65]:
#littleMelody = converter.parse(generated_melody)
#littleMelody.show('midi')

In [34]:
#saving
#littleMelody.write("midi", "testaudio.mkv")

# Try to merge different songs randomly 

In [124]:
st = ''
for i in range(10):
    l = dictionaries[i]
    keys = list(l.keys())
    generated_melody = generate_trigram(keys[0], 100, l)
    generated_melody = generated_melody[30:50]
    st += generated_melody

In [125]:
st = "tinynotation: 4/4" + st

In [126]:
littleMelody = converter.parse(st)
littleMelody.show('midi')